In [1]:
import open3d as o3d
import numpy as np
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('../sample')
from readdatas import *

In [3]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [4]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [5]:
ground = pcd.ground
ground_o3d = gpf.np_to_o3d(pcd.ground)

In [6]:
curvity = gpf.curvature_calculation(ground_o3d)

In [8]:
colors = gpf.colormap(curvity)

In [9]:
ground_o3d.colors = o3d.utility.Vector3dVector(colors)

In [10]:
o3d.visualization.draw_geometries([ground_o3d])